References

1.	http://excelsior-cjh.tistory.com/entry/5-Pandas를-이용한-Naver금융에서-주식데이터-가져오기
2.	https://financedata.github.io/posts/naver-finance-finstate-crawling.html
3.	https://gomjellie.github.io/파이썬/pandas/주식/2017/06/09/pandas-datareader-stock.html
4.	http://estenpark.tistory.com/353
5.	https://m.blog.naver.com/PostView.nhn?blogId=koo_m&logNo=221162347920&proxyReferer=https%3A%2F%2Fwww.google.com%2F
6.	http://blog.naver.com/PostView.nhn?blogId=suma_maple&logNo=220970555530
7.	http://dongsamb.com/web-scraping-using-python/
8.  https://github.com/datacamp/datacamp-community-tutorials
9.  https://www.datacamp.com/community/tutorials/finance-python-trading
10. https://www.scrapehero.com/scrape-yahoo-finance-stock-market-data/
11. https://www.facebook.com/financedata/

In [1]:
import pandas as pd

In [2]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
#download name and code in market 

In [3]:
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)

In [4]:
code_df = code_df[['회사명', '종목코드']]

In [5]:
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

In [6]:
code_df.head()

,name,code
0,CJ CGV,079160
1,CJ씨푸드,011150
2,DB손해보험,005830
3,DSR제강,069730
4,LS전선아시아,229640


In [7]:
def get_url(item_name, code_df):
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    url = "http://finance.naver.com/item/sise_day.nhn?code={code}".format(code=code)
    
    print("요청 URL = {}".format(url))
    return url

item_name = '카카오'
url = get_url(item_name, code_df)
#get precious data

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=035720


In [8]:
df = pd.DataFrame()

In [9]:
for page in range(1, 50):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
#pages previously

In [10]:
df = df.dropna()

In [11]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2018.02.01,141500,1500,140000,142000,137500,908229
1,2018.01.31,140000,1500,137500,140000,135000,943362
2,2018.01.30,138500,500,139000,140500,137500,642199
3,2018.01.29,139000,1000,141000,141500,138500,708612
4,2018.01.26,140000,5000,136000,141000,135000,1271640


In [12]:
df = df.rename(columns={'날짜': 'date', '종가': 'close', '전일비': 'diff',
                       '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

In [13]:
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
 = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) # 1000 = 1 d

In [14]:
df['date'] = pd.to_datetime(df['date'])

In [15]:
df = df.sort_values(by=['date'], ascending=True)

In [16]:
df.head()

,date,close,diff,open,high,low,volume
489,2016-02-03,109200,500,107900,109700,107900,220630
488,2016-02-04,109300,100,109200,109900,108700,145694
487,2016-02-05,108900,400,108200,112100,107300,372885
486,2016-02-11,100700,8200,104500,104900,100700,736337
485,2016-02-12,92800,7900,99000,100400,92700,986668


In [28]:
#pip install plotly
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode(connected=True)

trace = go.Scatter(x=df.date, y=df.close, name=item_name)

data = [trace]

layout = dict(
            title='{}의 종가(close) Time Series'.format(item_name),
            xaxis =dict(
                rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label='1m',
                        step='month',
                        stepmode='backward'),
                    dict(count=3,
                        label='3m',
                        step='month',
                        stepmode='backward'),
                    dict(count=6,
                        label='6m',
                        step='month',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(),
            type='date'
        )
    )

fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)

In [25]:
import logging
import pymongo
import crawlib
import time
from datetime import datetime
from gevent import monkey
monkey.patch_all()
from gevent.pool import Pool

collection = pymongo.MongoClient().stock.naver_discussion

ModuleNotFoundError: No module named 'pymongo'

In [26]:
class NaverCrawling(crawlib.Crawling):
    def get_item_list(self, cate, page):
        url = self.make_target_url(cate, page)
        soup = crawlib.getSoup(url)
        link_list = []
        for i in soup.find_all('tr',{'onmouseover':'mouseOver(this)'}):
            link = i.find_all('td')[1].a['href'].split('&st=&sw=&page')[0]
            link = crawlib.urljoin('http://finance.naver.com',link)

            # already exist this data
            if self.data_dic.has_key(link):
                print "exist", link
                continue

            self.data_dic[link] = {}
            self.data_dic[link]['date'] = i.find_all('td')[0].text.strip()
            self.data_dic[link]['title'] = i.find_all('td')[1].a['title'].strip()
            self.data_dic[link]['opinion'] = i.find_all('td')[2].text.strip()
            self.data_dic[link]['view'] = i.find_all('td')[4].text.strip()
            self.data_dic[link]['sympathy'] = i.find_all('td')[5].text.strip()
            self.data_dic[link]['unsympathy'] = i.find_all('td')[6].text.strip()
            self.data_dic[link]['_id'] = link

            link_list.append(link)
            # self.get_item(link)

        if len(link_list) == 0:
            return 'stop'

        pool = Pool(len(link_list))
        res = pool.map(self.get_item, link_list)

    def get_item(self, url, key=''):
        if not key:
            key = url
        soup = crawlib.getSoup(url)
        content = '\n'.join([i.text for i in soup.find_all('div','view_se')]).strip()
        user = soup.find('th','info').find('span').text.strip()
        nid = crawlib.get_param_from_url(url, 'nid')
        code = crawlib.get_param_from_url(url, 'code')

        self.data_dic[key]['content'] = content
        self.data_dic[key]['nid'] = nid
        self.data_dic[key]['code'] = code
        self.data_dic[key]['user'] = user

        print key, self.data_dic[key]['opinion']
        # runtime_check = time.time()
        collection.replace_one({"_id": key}, self.data_dic[key], upsert=True)
        # print time.time() - runtime_check


    # todo: mongo db library 화
    # todo: 최신글부터 가져오되 몇개이상(한페이지이상)  중복 감지시 stop'''

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-26-3cc51073f926>, line 12)

In [27]:
target_name = 'naver_stock_discussion'
naver_dicussion = NaverCrawling(target_name=target_name, base_url='http://finance.naver.com/item/board.nhn?', cate_param='code',
                                page_param='page', cate_list_path='category.txt', page_list=range(1, 1000))


naver_dicussion.crawling_cate_list(cate_delay=0, page_delay=0)
# 결과는 console 에서 확인 '''

NameError: name 'NaverCrawling' is not defined